In [1]:
import pandas as pd
import joblib

In [2]:
df=pd.read_csv('C://Users//nithin goud//Downloads//Data.csv')
df.head()

,Survey_Date,Age,Gender,Region,Industry,Job_Role,Work_Arrangement,Hours_Per_Week,Mental_Health_Status,Burnout_Level,Work_Life_Balance_Score,Physical_Health_Issues,Social_Isolation_Score,Salary_Range
0,2025-06-01,27,Female,Asia,Professional Services,Data Analyst,Onsite,64,Stress Disorder,High,3,Shoulder Pain; Neck Pain,2,$40K-60K
1,2025-06-01,37,Female,Asia,Professional Services,Data Analyst,Onsite,37,Stress Disorder,High,4,Back Pain,2,$80K-100K
2,2025-06-01,32,Female,Africa,Education,Business Analyst,Onsite,36,ADHD,High,3,Shoulder Pain; Eye Strain,2,$80K-100K
3,2025-06-01,40,Female,Europe,Education,Data Analyst,Onsite,63,ADHD,Medium,1,Shoulder Pain; Eye Strain,2,$60K-80K
4,2025-06-01,30,Male,South America,Manufacturing,DevOps Engineer,Hybrid,65,NaN,Medium,5,NaN,4,$60K-80K


In [3]:
df.columns

Index(['Survey_Date', 'Age', 'Gender', 'Region', 'Industry', 'Job_Role',
       'Work_Arrangement', 'Hours_Per_Week', 'Mental_Health_Status',
       'Burnout_Level', 'Work_Life_Balance_Score', 'Physical_Health_Issues',
       'Social_Isolation_Score', 'Salary_Range'],
      dtype='object')

In [6]:
model=joblib.load('random_forest.pkl')
burn=joblib.load('burn_map.pkl')
salary=joblib.load('salary_order.pkl')
health=joblib.load('health_pkl')
nom=joblib.load('nominal.pkl')
label=joblib.load('lable.pkl')
nominal_enc=joblib.load('dummy_columns.pkl')

In [12]:
import pandas as pd
import joblib
from sklearn.preprocessing import MultiLabelBinarizer

# --- Load models & encoders ---
model = joblib.load("C://Users//nithin goud//Mental Health Project//random_forest.pkl")
burn_map = joblib.load("C://Users//nithin goud//Mental Health Project//burn_map.pkl")
salary_enc = joblib.load("C://Users//nithin goud//Mental Health Project//salary_order.pkl")
label_enc = joblib.load("C://Users//nithin goud//Mental Health Project//lable.pkl")
nominal_enc = joblib.load("C://Users//nithin goud//Mental Health Project//health_pkl")
nominal_enc = joblib.load("C://Users//nithin goud//Mental Health Project//nominal.pkl")
dumy_enc = joblib.load("C://Users//nithin goud//Mental Health Project//dummy_columns.pkl")

# --- Load data for dropdown references ---
inp = pd.read_csv("C://Users//nithin goud//Downloads//mental.csv")

while True:
    print("\n--- Mental Health Status Prediction ---")
    age = eval(input(f"Enter Age: Range -> {df.Age.min()} to {df.Age.max()}"))
    gender = input(f"Gender {list(inp.Gender.unique())}: ")
    region = input(f"Region {list(inp.Region.unique())}: ")
    industry = input(f"Industry {list(inp.Industry.unique())}: ")
    job_role = input(f"Job Role {list(inp.Job_Role.unique())}: ")
    work_arrangement = input(f"Work Arrangement {list(inp.Work_Arrangement.unique())}: ")
    hours_per_week = int(input("Hours per Week: "))
    burnout_level = input(f"Burnout Level {list(inp.Burnout_Level.unique())}: ")
    work_life_balance = int(input("Work Life Balance (1-5): "))
    health_issues = input(f"Physical Health Issues{list(inp.Physical_Health_Issues.unique())}: ")
    social_isolation_score = int(input("Social Isolation Score (1-5): "))
    salary_range = input(f"Salary Range {list(inp.Salary_Range.unique())}: ")

    # Create DataFrame
    input_df = pd.DataFrame([[
        age, gender, region, industry, job_role, work_arrangement, hours_per_week,
        burnout_level, work_life_balance, health_issues, social_isolation_score, salary_range
    ]], columns=inp.columns)

    # Encode burnout
    input_df['Burnout_Level'] = burn_map.transform(input_df[['Burnout_Level']]) if hasattr(burn_map, 'transform') else input_df['Burnout_Level'].map(burn_map)

    # Encode salary
    input_df['Salary_Range'] = salary_enc.transform(input_df[['Salary_Range']]) if hasattr(salary_enc, 'transform') else input_df['Salary_Range'].map(salary_enc)

    # Encode health
    input_df['Physical_Health_Issues'] = input_df['Physical_Health_Issues'].apply(lambda x: x.split(',') if isinstance(x, str) and x.lower() != 'none' else [])
    mlb = MultiLabelBinarizer(classes=list(health_classes))
    mlb.fit([list(health_classes)])
    health_array = mlb.transform(input_df['Physical_Health_Issues'])
    health_df = pd.DataFrame(health_array, columns=mlb.classes_, index=input_df.index)
    input_df = pd.concat([input_df.drop('Physical_Health_Issues', axis=1), health_df], axis=1)

    # Encode nominal
    nominal_cols = ['Gender', 'Region', 'Industry', 'Job_Role', 'Work_Arrangement']
    nominal_data = input_df[nominal_cols]
    nominal_encoded = nominal_enc.transform(nominal_data)
    if hasattr(nominal_encoded, "toarray"):
        nominal_encoded = nominal_encoded.toarray()
    nominal_df = pd.DataFrame(nominal_encoded, columns=nominal_enc.get_feature_names_out(nominal_cols))

    input_df = input_df.drop(columns=nominal_cols)
    input_df = pd.concat([input_df.reset_index(drop=True), nominal_df.reset_index(drop=True)], axis=1)

    # Match features
    for col in model.feature_names_in_:
        if col not in input_df.columns:
            input_df[col] = 0
    input_df = input_df[model.feature_names_in_]

    # Predict
    prediction_encoded = model.predict(input_df)
    prediction = label_enc.inverse_transform(prediction_encoded) if hasattr(label_enc, 'inverse_transform') else prediction_encoded

    # Predict probabilities
    probabilities = model.predict_proba(input_df)[0]
    class_labels = label_enc.inverse_transform(range(len(probabilities)))
    proba_df = pd.DataFrame({"Class": class_labels, "Probability": probabilities})

    print(f"\n✅ Predicted Mental Health Status: {prediction[0]}")
    print("\n📊 Probabilities:")
    print(proba_df.to_string(index=False))



--- Mental Health Status Prediction ---
Enter Age: Range -> 22 to 6550
Gender ['Female', 'Male', 'Non-binary', 'Prefer not to say']: Male
Region ['Asia', 'Africa', 'Europe', 'South America', 'Oceania', 'North America']: Asia
Industry ['Professional Services', 'Education', 'Manufacturing', 'Customer Service', 'Technology', 'Finance', 'Retail', 'Healthcare', 'Marketing']: Education
Job Role ['Data Analyst', 'Business Analyst', 'DevOps Engineer', 'IT Support', 'Technical Writer', 'Software Engineer', 'HR Manager', 'Project Manager', 'Data Scientist', 'UX Designer', 'Social Media Manager', 'Digital Marketing Specialist', 'Account Manager', 'Research Scientist', 'Sales Representative', 'Operations Manager', 'Customer Service Manager', 'Product Manager', 'Quality Assurance', 'Executive Assistant', 'Financial Analyst', 'Marketing Specialist', 'Consultant', 'Content Writer']: Marketing Specialist
Work Arrangement ['Onsite', 'Hybrid', 'Remote']: Onsite
Hours per Week: 5
Burnout Level ['High', 

SpecificationError: nested renamer is not supported

In [8]:
Age:45
Gender: "Female"
Region:"Africa",
Industry: "Technology",
Job_Role: "Software Engineer",
Work_Arrangement: "Remote",
Hours_Per_Week: 40,
Burnout_Level: "High",
Work_Life_Balance_Score: 5,
Physical_Health_Issues: "Back Pain; Neck Pain",
Social_Isolation_Score: 3,
Salary_Range: "$40K-60K",

SyntaxError: invalid syntax (825018559.py, line 3)

In [9]:
def mental_condition():
    print("Model Built on Below Input Data:")
    display(df.head())
    # User Input
    print("Enter Details One by One To Get Outcome:")
    age = eval(input(f"Enter Age: Range -> {df.Age.min()} to {df.Age.max()}"))
    print()
    print(df.Gender.unique())
    Gender=input("Select Gender:")
    print()
    print(df.Region.unique())
    Region=input("Select Region:")
    print()
    print(df.Industry.unique())
    Industry=input("Select Industry:")
    print()
    print(df.Job_Role.unique())
    Job_Role=input("Select Job_Role:")
    print()
    print(df.Work_Arrangement.unique())
    Work_Arrangement=input("select Work_Arrangement:")
    print()
    hpw = eval(input(f"Enter Hours_Per_Week: Range -> {df.Hours_Per_Week.min()} to {df.Hours_Per_Week.max()}"))
    print()
    print(df.Burnout_Level.unique())
    burn = input("Select Burnout_Level:")
    print()
    wls = eval(input(f"Enter Work_Life_Balance_Score: Range -> {df.Work_Life_Balance_Score.min()} to {df.Work_Life_Balance_Score.max()}"))
    print()
    print(df.Physical_Health_Issues.unique())
    health=input("Select Physical_Health_Issues:")
    print()
    sis = eval(input(f"Enter Social_Isolation_Score: Range -> {df.Social_Isolation_Score.min()} to {df.Social_Isolation_Score.max()}"))
    print()
    print(df.Salary_Range.unique())
    salary=input("Select Salary_Range:")
    print()
    row = pd.DataFrame([[age,Gender,Region,Industry,Job_Role,Work_Arrangement,hpw,burn,wls,health,sis,salary]], columns=df.columns)
    print("Given Data:")
    display(row)
###Pre processing
    row.replace({'Low': 3, 'Medium': 1, 'High': 2},inplace=True)
    row.rerplace({'$40K-60K': 3, '$80K-100K': 2,'$60K-80K':1,'$100K-120K': 4,'$120K+':5},inplace=True)
    row=
    
    # Prediction
    print("Health:", model.classes_)
    print("Probabilities:",[round(prob,2) for prob in model.predict_proba(row)[0]])
    print("Predicted Mental Health Condition:", model.predict(row)[0])

SyntaxError: invalid syntax (3463383830.py, line 44)